In [94]:
from google.colab import drive, files
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Upload kaggle api key
! pip install -q kaggle
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Fetch train and test data from kaggle
! kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

! mkdir train
! unzip train.zip -d train > downloadlog.txt

! mkdir test
! unzip test.zip -d test >> downloadlog.txt

In [ ]:
import glob
import os
import cv2
import math
import datetime
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shutil

from PIL import Image
from tqdm import tqdm
from pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss

from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.callbacks import Callback
from tensorflow import keras

In [95]:
# Check dirs on VM
os.listdir()

['.config',
 'drive',
 'kaggle.json',
 'data',
 'train',
 'test.zip',
 'downloadlog.txt',
 'test',
 'sample_submission.csv',
 'train.zip',
 'sample_data']

In [ ]:
# Import UDFs
os.chdir('drive/My Drive/collab')
from helper_functions import *

In [ ]:
# Settings
INITIAL_TRAIN_DIR = 'train/train/' 
COMP_DIR = 'test/test/' 

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_data_dir = 'data/test'

np.random.seed(0)
tf.random.set_seed(0)

ROWS = 224
COLS = 224
CHANNELS = 3

MIN_LR = 0.00002
MAX_LR = 0.0002 
BATCHSIZE = 32
CLR_METHOD = "triangular"
GAMMA = 0.99995
EPOCHS = 8 #64
STEP_SIZE = 4 

rotation_range = 15
zoom_range = 0.15
width_shift_range = 0.2
height_shift_range = 0.2
shear_range = 0.15
horizontal_flip = True
fill_mode = "nearest"

experiment_name = 'resnet_imagenet'
model_version = str(13)

rcParams['figure.figsize'] = 30, 6

In [ ]:
os.chdir('../../..')
X_files = np.array([INITIAL_TRAIN_DIR+i for i in os.listdir(INITIAL_TRAIN_DIR)])

# Get train and testset
X_train_filenames, X_test_filenames = train_test_split(X_files, test_size=0.1, random_state=0)

# Split traindata again into train and validation
X_train_filenames, X_val_filenames = train_test_split(X_train_filenames, test_size=0.1, random_state=0)

In [ ]:
sort_test_images(COMP_DIR)
prepare_file_structure(X_train_filenames, X_val_filenames, X_test_filenames)
show_directory_structure(startpath='data')

In [ ]:
# Show examples of filenames
os.listdir('data/train/dogs/')[:5]

In [97]:
base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, pooling = 'avg', input_shape=(ROWS, COLS, CHANNELS), classes=2)
base_model.trainable = False

inputs = keras.Input(shape=(ROWS, COLS, CHANNELS))
x = base_model(inputs, training=False)
# x = AveragePooling2D(pool_size=(2,2), padding='same')(x)
# x = Flatten()(x)
# x = Dense(256, activation="relu")(x)
# x = Dropout(0.25)(x)
outputs = keras.layers.Dense(2, activation='softmax')(x)
model = keras.Model(inputs, outputs)

model.layers[0].trainable = False 

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999),
              loss='sparse_categorical_crossentropy',  # Sparse for integers, just categorical for one hot encoded
              metrics=['accuracy'])

model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [98]:
for i in model.layers:
    print(i.trainable)

False
False
True


In [ ]:
# save best model checkpoint
save_model = tf.keras.callbacks.ModelCheckpoint('drive/My Drive/collab/' + experiment_name + '.h5', monitor='val_loss', verbose=1, save_best_only=True, 
                                                save_weights_only=False, mode='auto', save_freq='epoch')

# Early stopping callback
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')

# Cyclical learning rate callback
clr = CyclicLR(base_lr=MIN_LR, max_lr=MAX_LR, step_size=STEP_SIZE * -(-len(X_train_filenames)//BATCHSIZE), mode=CLR_METHOD, gamma=GAMMA)

# Create tensorboard callback
log_dir = "drive/My Drive/collab/logs/fit/" + experiment_name
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Create augmentation generator
train_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
		#rescale=1./255,
    rotation_range=rotation_range,
		zoom_range=zoom_range,
		width_shift_range=width_shift_range,
		height_shift_range=height_shift_range,
		shear_range=shear_range,
		horizontal_flip=horizontal_flip,
		fill_mode=fill_mode
)

test_aug = ImageDataGenerator(preprocessing_function=preprocess_input
                              #rescale=1./255
                              )

train_generator = train_aug.flow_from_directory(train_data_dir, target_size=(ROWS, COLS), batch_size=BATCHSIZE, class_mode='binary')
val_generator = test_aug.flow_from_directory(validation_data_dir, target_size=(ROWS, COLS), batch_size=BATCHSIZE, class_mode='binary')
test_generator = test_aug.flow_from_directory(test_data_dir, target_size=(ROWS, COLS), batch_size=BATCHSIZE, class_mode='binary', shuffle=False)
comp_generator = test_aug.flow_from_directory('test', target_size=(ROWS, COLS), batch_size=BATCHSIZE, class_mode='binary', shuffle=False)

In [99]:
# Train model
start = datetime.datetime.now()
print(start)

history = model.fit(train_generator,
                    batch_size=BATCHSIZE,
                    epochs=1, 
                    validation_data=val_generator,
                    verbose=1,
                    callbacks=[early, save_model, tensorboard_callback] #lr_callback    clr,
                ) 

print('Finished in: {}'.format(datetime.datetime.now() - start))
print(datetime.datetime.now())

2020-11-12 11:03:24.860109
633/633 [==============================] - ETA: 0s - loss: 0.1912 - accuracy: 0.9691
Epoch 00001: val_loss did not improve from 0.06824
633/633 [==============================] - 294s 464ms/step - loss: 0.1912 - accuracy: 0.9691 - val_loss: 0.1040 - val_accuracy: 0.9867
Finished in: 0:04:59.146386
2020-11-12 11:08:24.006654


In [100]:
model.optimizer.learning_rate.numpy()

0.01

In [101]:
K.set_value(model.optimizer.learning_rate, 0.001)

In [102]:
model.optimizer.learning_rate.numpy()

0.001

In [103]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_17[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [104]:
len(base_model.layers)

176

In [123]:
model.layers[1].trainable = True

In [127]:
for i in range(len(base_model.layers)):
    if not base_model.layers[i].name.startswith('conv5'):
        base_model.layers[i].trainable = False

In [128]:
for l in base_model.layers:
    print(l.name, l.trainable)

input_17 False
conv1_pad False
conv1_conv False
conv1_bn False
conv1_relu False
pool1_pad False
pool1_pool False
conv2_block1_1_conv False
conv2_block1_1_bn False
conv2_block1_1_relu False
conv2_block1_2_conv False
conv2_block1_2_bn False
conv2_block1_2_relu False
conv2_block1_0_conv False
conv2_block1_3_conv False
conv2_block1_0_bn False
conv2_block1_3_bn False
conv2_block1_add False
conv2_block1_out False
conv2_block2_1_conv False
conv2_block2_1_bn False
conv2_block2_1_relu False
conv2_block2_2_conv False
conv2_block2_2_bn False
conv2_block2_2_relu False
conv2_block2_3_conv False
conv2_block2_3_bn False
conv2_block2_add False
conv2_block2_out False
conv2_block3_1_conv False
conv2_block3_1_bn False
conv2_block3_1_relu False
conv2_block3_2_conv False
conv2_block3_2_bn False
conv2_block3_2_relu False
conv2_block3_3_conv False
conv2_block3_3_bn False
conv2_block3_add False
conv2_block3_out False
conv3_block1_1_conv False
conv3_block1_1_bn False
conv3_block1_1_relu False
conv3_block1_2_co

In [129]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_17[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [130]:
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 14,980,098
Non-trainable params: 8,611,712
_________________________________________________________________


In [131]:
# Train model
start = datetime.datetime.now()
print(start)

history2 = model.fit(train_generator,
                    batch_size=BATCHSIZE,
                    epochs=5, 
                    validation_data=val_generator,
                    verbose=1,
                    callbacks=[early, save_model, tensorboard_callback] #lr_callback    clr,
                ) 

print('Finished in: {}'.format(datetime.datetime.now() - start))
print(datetime.datetime.now())

2020-11-12 11:20:31.649645
Epoch 1/5
633/633 [==============================] - ETA: 0s - loss: 0.1024 - accuracy: 0.9851
Epoch 00001: val_loss did not improve from 0.06824
633/633 [==============================] - 294s 464ms/step - loss: 0.1024 - accuracy: 0.9851 - val_loss: 0.0827 - val_accuracy: 0.9876
Epoch 2/5
633/633 [==============================] - ETA: 0s - loss: 0.0759 - accuracy: 0.9864
Epoch 00002: val_loss improved from 0.06824 to 0.06724, saving model to drive/My Drive/collab/resnet_imagenet.h5
633/633 [==============================] - 294s 465ms/step - loss: 0.0759 - accuracy: 0.9864 - val_loss: 0.0672 - val_accuracy: 0.9893
Epoch 3/5
633/633 [==============================] - ETA: 0s - loss: 0.0692 - accuracy: 0.9868
Epoch 00003: val_loss improved from 0.06724 to 0.05139, saving model to drive/My Drive/collab/resnet_imagenet.h5
633/633 [==============================] - 294s 465ms/step - loss: 0.0692 - accuracy: 0.9868 - val_loss: 0.0514 - val_accuracy: 0.9907
Epoch 

In [132]:
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

79/79 [==============================] - 10s 131ms/step - loss: 0.0675 - accuracy: 0.9888


In [133]:
model.optimizer.learning_rate.numpy()

0.001

In [134]:
K.set_value(model.optimizer.learning_rate, 0.0001)

In [135]:
model.optimizer.learning_rate.numpy()

1e-04

In [136]:
model.layers[1].trainable = True

In [137]:
for i in range(len(base_model.layers)):
    if not base_model.layers[i].name.startswith('conv5') and not base_model.layers[i].name.startswith('conv4'):
        base_model.layers[i].trainable = False

In [138]:
for l in base_model.layers:
    print(l.name, l.trainable)

input_17 False
conv1_pad False
conv1_conv False
conv1_bn False
conv1_relu False
pool1_pad False
pool1_pool False
conv2_block1_1_conv False
conv2_block1_1_bn False
conv2_block1_1_relu False
conv2_block1_2_conv False
conv2_block1_2_bn False
conv2_block1_2_relu False
conv2_block1_0_conv False
conv2_block1_3_conv False
conv2_block1_0_bn False
conv2_block1_3_bn False
conv2_block1_add False
conv2_block1_out False
conv2_block2_1_conv False
conv2_block2_1_bn False
conv2_block2_1_relu False
conv2_block2_2_conv False
conv2_block2_2_bn False
conv2_block2_2_relu False
conv2_block2_3_conv False
conv2_block2_3_bn False
conv2_block2_add False
conv2_block2_out False
conv2_block3_1_conv False
conv2_block3_1_bn False
conv2_block3_1_relu False
conv2_block3_2_conv False
conv2_block3_2_bn False
conv2_block3_2_relu False
conv2_block3_3_conv False
conv2_block3_3_bn False
conv2_block3_add False
conv2_block3_out False
conv3_block1_1_conv False
conv3_block1_1_bn False
conv3_block1_1_relu False
conv3_block1_2_co

In [139]:
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 22,088,706
Non-trainable params: 1,503,104
_________________________________________________________________


In [140]:
# Train model
start = datetime.datetime.now()
print(start)

history3 = model.fit(train_generator,
                    batch_size=BATCHSIZE,
                    epochs=10, 
                    validation_data=val_generator,
                    verbose=1,
                    callbacks=[early, save_model, tensorboard_callback] #lr_callback    clr,
                ) 

print('Finished in: {}'.format(datetime.datetime.now() - start))
print(datetime.datetime.now())

2020-11-12 11:47:06.924442
Epoch 1/10
633/633 [==============================] - ETA: 0s - loss: 0.0410 - accuracy: 0.9898
Epoch 00001: val_loss did not improve from 0.04105
633/633 [==============================] - 292s 461ms/step - loss: 0.0410 - accuracy: 0.9898 - val_loss: 0.0425 - val_accuracy: 0.9916
Epoch 2/10
633/633 [==============================] - ETA: 0s - loss: 0.0442 - accuracy: 0.9899
Epoch 00002: val_loss did not improve from 0.04105
633/633 [==============================] - 291s 460ms/step - loss: 0.0442 - accuracy: 0.9899 - val_loss: 0.0445 - val_accuracy: 0.9911
Epoch 3/10
633/633 [==============================] - ETA: 0s - loss: 0.0460 - accuracy: 0.9891
Epoch 00003: val_loss did not improve from 0.04105
633/633 [==============================] - 290s 458ms/step - loss: 0.0460 - accuracy: 0.9891 - val_loss: 0.0415 - val_accuracy: 0.9916
Epoch 4/10
633/633 [==============================] - ETA: 0s - loss: 0.0404 - accuracy: 0.9898
Epoch 00004: val_loss did not i

In [ ]:
# loss: 0.0510 - accuracy: 0.9887 - val_loss: 0.0411 - val_accuracy: 0.9929

# loss: 0.0675 - accuracy: 0.9888

In [143]:
# read in best version of model here
saved_model = tf.keras.models.load_model('drive/My Drive/collab/' + experiment_name + '.h5')

In [144]:
test_loss, test_acc = saved_model.evaluate(test_generator, verbose=1)

79/79 [==============================] - 10s 130ms/step - loss: 0.0553 - accuracy: 0.5060


In [145]:
preds = []
actuals = []
for i in range(78):
    X, y = test_generator.next()
    prediction = saved_model.predict(X)
    preds.append(list(np.round(prediction[:, 1])))
    actuals.append(list(y))

actuals = [item for sublist in actuals for item in sublist]
preds = [item for sublist in preds for item in sublist]
print('Accuracy score: {}'.format(accuracy_score(actuals, preds)))
confusion_matrix(actuals, preds)

Accuracy score: 0.9895833333333334


array([[1251,   12],
       [  14, 1219]])

In [ ]:
# Load some images to see if everything went right
test_images = [COMP_DIR + x for x in os.listdir(COMP_DIR)]
test_images.sort()
comp_data = prep_data(test_images[:9], ROWS, COLS, CHANNELS)

fig, axes = plt.subplots(3, 3, figsize=(12, 12))

for i, ax in enumerate(axes.flat):
    ax.imshow(comp_data[i])

In [146]:
predictions_comp_data = pd.DataFrame({'id': range(1, len(os.listdir(COMP_DIR)) + 1), 'label': np.clip(saved_model.predict(comp_generator)[:, 1], 0.02, 0.98)})
predictions_comp_data['label'] = round(predictions_comp_data['label'], 4)
predictions_comp_data.head(9)

,id,label
0,1,0.98
1,2,0.98
2,3,0.98
3,4,0.98
4,5,0.02
5,6,0.02
6,7,0.02
7,8,0.02
8,9,0.02


In [147]:
submission_path = 'drive/My Drive/collab/submission_transfer_resnet50_clip02' + '.csv'
predictions_comp_data.to_csv(submission_path, index=False)

In [148]:
! kaggle competitions submit -c dogs-vs-cats-redux-kernels-edition -f 'drive/My Drive/collab/submission_transfer_resnet50_clip02.csv' -m "resnet50 transfer imagenet retraining top, clip 0.02"

100% 124k/124k [00:04<00:00, 26.0kB/s]
Successfully submitted to Dogs vs. Cats Redux: Kernels Edition

In [149]:
174/1214

0.14332784184514002